# Testing how user data will be stored, processed and made ready for the javascript code running on the client to generate graphs with.

In [1]:
import pandas as pd
import json
from random import randint
from time import time


## Database is loaded in memory when the server starts up


In [2]:
database = pd.DataFrame()

## Generating some random test data

In [3]:

def get_random_raw():
    
    """
    This is similar to the format provided by the html-form object.
    """    
    test_names = [f'0{i}' for i in range(10)]
    orig_names = [f'{n}_0' for n in test_names]
    impaired1_names = [f'{n}_1' for n in test_names]
    impaired2_names = [f'{n}_2' for n in test_names]
    names = orig_names + impaired1_names+ impaired2_names
    raw_data = [(name, randint(1, 5)) for name in names]
    # timestamp to identify user (since we're not gonna use emails)
    raw_data.insert(0, ("user", int(time()*1000)))
    return raw_data

In [4]:
get_random_raw()

[('user', 1640620238093),
 ('00_0', 3),
 ('01_0', 1),
 ('02_0', 1),
 ('03_0', 4),
 ('04_0', 3),
 ('05_0', 3),
 ('06_0', 2),
 ('07_0', 2),
 ('08_0', 4),
 ('09_0', 1),
 ('00_1', 3),
 ('01_1', 5),
 ('02_1', 5),
 ('03_1', 2),
 ('04_1', 4),
 ('05_1', 1),
 ('06_1', 1),
 ('07_1', 4),
 ('08_1', 1),
 ('09_1', 5),
 ('00_2', 3),
 ('01_2', 3),
 ('02_2', 1),
 ('03_2', 1),
 ('04_2', 2),
 ('05_2', 2),
 ('06_2', 4),
 ('07_2', 4),
 ('08_2', 1),
 ('09_2', 3)]

In [5]:
def raw_data_to_record(raw_data):
    """
    Convert raw data from the form to a record (row). 
    """
    new_record = pd.DataFrame(raw_data).T
    new_record = new_record.rename(columns=new_record.iloc[0])[1:]
    return new_record

# This function will be called at each time a new user submits their form:


In [6]:
def append_to_database(raw_data):
    """
    Append raw data from a new user to the database
    """
    global database
    new_record = raw_data_to_record(raw_data)
    database = database.append(new_record).reset_index(drop=True)

## Call it n times to append new random data from n 'users'

In [15]:
append_to_database(get_random_raw())

In [16]:
database

,user,00_0,01_0,02_0,03_0,04_0,05_0,06_0,07_0,08_0,...,00_2,01_2,02_2,03_2,04_2,05_2,06_2,07_2,08_2,09_2
0,1640620249599,5,5,5,3,3,2,2,1,4,...,5,1,1,5,1,5,5,1,1,1
1,1640620250996,4,2,5,3,1,4,1,1,2,...,5,4,2,1,4,4,2,2,1,5
2,1640620251255,3,4,5,1,4,4,5,4,5,...,5,2,3,4,5,1,2,1,5,1
3,1640620251487,3,3,1,1,1,2,2,4,4,...,3,4,5,1,1,1,3,5,5,4
4,1640620251728,5,3,1,2,4,5,2,4,4,...,2,2,5,1,3,1,3,4,1,4
5,1640620251958,2,2,3,1,3,2,4,3,4,...,5,3,1,3,4,1,3,5,4,5
6,1640620252185,4,4,1,4,2,5,5,2,3,...,2,3,4,4,2,3,4,5,3,3
7,1640620252404,4,5,1,2,1,1,1,3,4,...,3,4,5,2,5,1,2,1,3,2
8,1640620252710,3,2,5,4,1,2,5,3,3,...,2,2,4,5,3,5,3,3,2,2


In [372]:
#database

## Calculating mos and mos-stddev values

In [17]:
class ImageTypes:
    ORIGINAL = "_0"
    IMPAIRED1 = "_1"
    IMPAIRED2 = "_2"

In [18]:
def get_names():
    return [col[:2] for col in database.columns if ImageTypes.ORIGINAL in col ]

In [19]:
def get_mos(img_type):
    cols = [col for col in database.columns if img_type in col ]
    return database[cols].mean(axis=0).to_list()

In [20]:
def get_mos_stdev(img_type):
    cols = [col for col in database.columns if img_type in col ]
    return database[cols].std(axis=0).to_list()

### Eg: mos of original images:

In [21]:
get_mos(ImageTypes.ORIGINAL)

[3.6666666666666665,
 3.3333333333333335,
 3.0,
 2.3333333333333335,
 2.2222222222222223,
 3.0,
 3.0,
 2.7777777777777777,
 3.6666666666666665,
 3.2222222222222223]

### Eg: stdev of mos of impaired2 images

In [22]:
get_mos_stdev(ImageTypes.IMPAIRED2)

[1.4240006242195884,
 1.092906420717,
 1.6583123951777,
 1.69148192751537,
 1.536590742882148,
 1.8104634152000358,
 1.0,
 1.8027756377319946,
 1.6414763002993509,
 1.5811388300841898]

# Api call functions:

In [23]:
def get_mos_json():
    result = {}
    result["names"] = get_names()
    result["original-image-mos"] = get_mos(ImageTypes.ORIGINAL)
    result["slightly-impaired-image-mos"] = get_mos(ImageTypes.IMPAIRED1)
    result["heavily-impaired-image-mos"] = get_mos(ImageTypes.IMPAIRED2)
    return json.dumps(result)
    

In [24]:
def get_mos_stddev_json():
    result = {}
    result["names"] = get_names()
    result["original-image-mos"] = get_mos_stdev(ImageTypes.ORIGINAL)
    result["slightly-impaired-image-mos"] = get_mos_stdev(ImageTypes.IMPAIRED1)
    result["heavily-impaired-image-mos"] = get_mos_stdev(ImageTypes.IMPAIRED2)
    return json.dumps(result)
    

In [25]:
get_mos_json()

'{"names": ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"], "original-image-mos": [3.6666666666666665, 3.3333333333333335, 3.0, 2.3333333333333335, 2.2222222222222223, 3.0, 3.0, 2.7777777777777777, 3.6666666666666665, 3.2222222222222223], "slightly-impaired-image-mos": [2.888888888888889, 3.888888888888889, 2.888888888888889, 2.4444444444444446, 3.5555555555555554, 2.6666666666666665, 2.6666666666666665, 2.4444444444444446, 3.3333333333333335, 2.3333333333333335], "heavily-impaired-image-mos": [3.5555555555555554, 2.7777777777777777, 3.3333333333333335, 2.888888888888889, 3.111111111111111, 2.4444444444444446, 3.0, 3.0, 2.7777777777777777, 3.0]}'

In [26]:
get_mos_stddev_json()

'{"names": ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"], "original-image-mos": [1.0, 1.224744871391589, 2.0, 1.224744871391589, 1.3017082793177759, 1.5, 1.7320508075688772, 1.2018504251546631, 0.8660254037844386, 1.7873008824606014], "slightly-impaired-image-mos": [1.6158932858054431, 1.4529663145135578, 1.7638342073763937, 1.6666666666666667, 1.5898986690282426, 1.5811388300841898, 1.3228756555322954, 1.2360330811826103, 1.6583123951777001, 1.118033988749895], "heavily-impaired-image-mos": [1.4240006242195884, 1.092906420717, 1.6583123951777, 1.69148192751537, 1.536590742882148, 1.8104634152000358, 1.0, 1.8027756377319946, 1.6414763002993509, 1.5811388300841898]}'